In [1]:
from Siamese import *
import numpy as np
import sys
import imageio
from scipy import misc
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops


In [3]:
Imag_o = np.zeros([1,600,800,3])
Imag_c = np.zeros([1,600,800,3])
Imag_y = np.zeros([1,600,800])
ImgNameIndex = ['0002','0003','0008','0009','0087']
for i in range(1):
    ImgN = ImgNameIndex[int(i/5)]
    ImgV = '0'+str(i%5)
    Imag_o[i,:,:,:]=imageio.imread('data/NoShadow/'+'Scene'+ImgN+'_View'+ImgV+'_moving.png')/float(255)
    Imag_c[i,:,:,:]=imageio.imread('data/NoShadow/'+'Scene'+ImgN+'_View'+ImgV+'_target.png')/float(255)
    Imag_y[i,:,:] = misc.imread('data/GroundTruth/Scene'+ImgN+'_View'+ImgV+'_gtmask.png',flatten = True)/float(255)


/Users/prashantgavit/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.


In [4]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    
    X_o , X_c , Y = create_placeholder(600, 800, 3 )
    parameters = initialize_parameter()
    
    Xcnn_o = forward_propogation(X_o,parameters)
    Xcnn_c = forward_propogation(X_c,parameters)
    diff = get_Euclidian_Distnace(Xcnn_o, Xcnn_c)
    Lu = get_Lu(diff)
    Lc = get_Lc(diff)
    loss = get_loss(Y, Lu, Lc)
    
    init = tf.global_variables_initializer()
    
    sess.run(init)
    
    a = sess.run(loss, {X_o : Imag_o,
                        X_c : Imag_c, Y : Imag_y })

    print("Z5 = " ,a)

Z5 =  170.671


In [5]:
def model(Imag_o, Imag_c, Imag_y,learning_rate = 0.009, num_epochs = 100, print_cost = True):
    ops.reset_default_graph()
    
    (m , n_H0, n_W0, n_C0) = Imag_o.shape
    costs = []
    
    X_o , X_c, Y = create_placeholder(n_H0, n_W0, n_C0)
    
    parameters = initialize_parameter()
    
    Xcnn_o = forward_propogation(X_o,parameters)
    Xcnn_c = forward_propogation(X_c,parameters)
    diff = get_Euclidian_Distnace(Xcnn_o, Xcnn_c)
    Lu = get_Lu(diff)
    Lc = get_Lc(diff)
    loss = get_loss(Y, Lu, Lc)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        
        sess.run(init)
        
        for epoch in range(num_epochs):
                
            _, temp_cost = sess.run([optimizer,loss], feed_dict={X_o : Imag_o,X_c : Imag_c, Y : Imag_y })
            
            print(temp_cost)
                


In [ ]:
model(Imag_o,Imag_c,Imag_y)